## 수정사항
#### 2021.06.06
* T_ej는 마지막 조인트 각도에 영향을 받음, 하지만 Q는 unkown이므로 특정하는게 불가능, T_ee로 전부 새로 학습할 것.

In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import os
import shutil
import random
PROJ_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(PROJ_DIR, "src"))

from pkg.utils.utils_python3 import *
from pkg.planning.filtering.lattice_model.data_utils import *

DATA_PATH = os.path.join(PROJ_DIR, "data")
LAT_DATA_PATH = os.path.join(DATA_PATH, "latticized")
MODEL_PATH = os.path.join(PROJ_DIR, "model")
LAT_MODEL_PATH = os.path.join(MODEL_PATH,"latticized")
try_mkdir(MODEL_PATH)
try_mkdir(LAT_MODEL_PATH)
GRASP_FOLDER = "grasp"
ARM10_FOLDER = "arm_10"
ARM05_FOLDER = "arm_05"
FULLS_FOLDER = "full_scene"

In [3]:
ROBOT_TYPE_NAME="panda"
ROBOT_DATA_ROOT = os.path.join(LAT_DATA_PATH, ROBOT_TYPE_NAME)
ROBOT_DATA_ROOT_FAILMORE = ROBOT_DATA_ROOT+"-failmore"
ROBOT_MODEL_ROOT =  os.path.join(LAT_MODEL_PATH, ROBOT_TYPE_NAME)
ARM_FOLDER = ARM10_FOLDER

In [4]:
dataset_list = sorted(os.listdir(ROBOT_DATA_ROOT))
DATASET_TRAIN_FILTERED = dataset_list[:10]
DATASET_TEST_FILTERED = dataset_list[10:15]
print(DATASET_TRAIN_FILTERED)
print(DATASET_TEST_FILTERED)

dataset_list_fails = sorted([folder for folder 
                             in os.listdir(ROBOT_DATA_ROOT_FAILMORE) 
                             if not folder.startswith(".")])
DATASET_TRAIN_FAILMORE = dataset_list_fails[:10]
DATASET_TEST_FAILMORE = dataset_list_fails[10:15]
print(DATASET_TRAIN_FAILMORE)
print(DATASET_TEST_FAILMORE)

['20210920-194311', '20210920-233941', '20210921-032259', '20210921-072543', '20210921-111615', '20210921-150638', '20210921-192001', '20210921-232015', '20210922-033134', '20210922-074138']
['20210922-113744', '20210922-153855', '20210922-193840', '20210922-233844', '20210923-040005']
['20211003-121041', '20211003-123042', '20211003-124608', '20211003-130439', '20211003-132729', '20211003-134230', '20211003-140040', '20211003-141918', '20211003-144018', '20211003-150523']
['20211003-152628', '20211003-154656', '20211003-160916', '20211003-162800', '20211003-164854']


In [5]:
data_pairs_train_filtered = []
for dataset in DATASET_TRAIN_FILTERED:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_train_filtered.append((os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT, dataset, ARM_FOLDER, file)))
print("train set: {}".format(len(data_pairs_train_filtered)))        
        
        
data_pairs_test_filtered = []
for dataset in DATASET_TEST_FILTERED:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_test_filtered.append((os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT, dataset, ARM_FOLDER, file)))
print("test set: {}".format(len(data_pairs_test_filtered)))        

data_pairs_train_failmore = []
for dataset in DATASET_TRAIN_FAILMORE:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_train_failmore.append((os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, ARM_FOLDER, file)))
print("train fail set: {}".format(len(data_pairs_train_failmore)))        
        
        
data_pairs_test_failmore = []
for dataset in DATASET_TEST_FAILMORE:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_test_failmore.append((os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, ARM_FOLDER, file)))
print("test fail set: {}".format(len(data_pairs_test_failmore)))        

train set: 10003
test set: 5000
train fail set: 10026
test fail set: 5025


In [6]:
N_max_train = 10000
N_max_test = 5000

print("=== mix data ===")
data_pairs_train_mixed = mix_data_pairs(
    data_pairs_train_filtered, data_pairs_train_failmore, N_max_train)
data_pairs_test_mixed = mix_data_pairs(
    data_pairs_test_filtered, data_pairs_test_failmore, N_max_test)

data_pairs_train_filtered = data_pairs_train_filtered[:N_max_train]
data_pairs_test_filtered = data_pairs_test_filtered[:N_max_test]
data_pairs_train_failmore = data_pairs_train_failmore[:N_max_train]
data_pairs_test_failmore = data_pairs_test_failmore[:N_max_test]

print("="*20)
print("filtered data: {} / {}".format(
    len(data_pairs_train_filtered), 
    len(data_pairs_test_filtered)))
print("failmore data: {} / {}".format(
    len(data_pairs_train_failmore), 
    len(data_pairs_test_failmore)))
print("mixed data: {} / {}".format(
    len(data_pairs_train_mixed), 
    len(data_pairs_test_mixed)))

=== mix data ===
filtered success: 5000
success from failmore: 0
dataset fails: 5000
dataset all: 10000
filtered success: 2500
success from failmore: 0
dataset fails: 2500
dataset all: 5000
filtered data: 10000 / 5000
failmore data: 10000 / 5000
mixed data: 10000 / 5000


## Train

In [7]:
from pkg.planning.filtering.lattice_model.lattice_model import *

# Create an instance of the model
model = ResNetModelTP()

loss_object = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
    

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.BinaryAccuracy(name='test_accuracy')
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

    
def train_script(data_pairs_train, data_pairs_test, robot_model_root,
                 EPOCHS_S=0, EPOCHS_E=15, BATCH_SIZE=16, LOG_STEP=100):
    current_time = get_now()
    logpath = os.path.join(robot_model_root, current_time)
    try_mkdir(logpath)
    train_log_dir = os.path.join(logpath, 'train')
    test_log_dir = os.path.join(logpath, 'test')
    model_log_dir = os.path.join(logpath, 'model_{}/')
    train_summary_writer = tf.summary.create_file_writer(train_log_dir)
    test_summary_writer = tf.summary.create_file_writer(test_log_dir)
    shutil.copy(os.path.join(
        PROJ_DIR,'src', 'pkg','planning','filtering','lattice_model','lattice_model.py' ), logpath)
    print(f'Log path: {logpath}')
    
    N_train = len(data_pairs_train)
    N_test = len(data_pairs_test)
    random.shuffle(data_pairs_test)
    res_list = []
    
    for epoch in range(EPOCHS_S, EPOCHS_E):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

        random.shuffle(data_pairs_train)
        i_step = 0
        data_batch, label_batch = [], []
        for data_pair in data_pairs_train:
            i_step += 1
            grasp_img, arm_img, rh_mask, label = load_data(data_pair)
            data_batch.append([grasp_img, arm_img, rh_mask])
            label_batch.append(label)
            if i_step%BATCH_SIZE==0:
                grasp_img_batch = np.array([grasp_img for grasp_img, arm_img, rh_mask in data_batch])
                arm_img_batch = np.array([arm_img for grasp_img, arm_img, rh_mask in data_batch])
                rh_mask_batch = np.array([rh_mask for grasp_img, arm_img, rh_mask in data_batch])
                label_batch = np.array(label_batch, dtype=np.int)
                train_step([grasp_img_batch, arm_img_batch, rh_mask_batch], label_batch)
                data_batch, label_batch = [], []
            if i_step%LOG_STEP==0:
                print("train step - {}/{}        ".format(i_step, N_train), end = '\r')
        with train_summary_writer.as_default():
            tf.summary.scalar('loss', train_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

        i_step = 0
        data_batch, label_batch = [], []
        for data_pair in data_pairs_test:
            i_step += 1
            grasp_img, arm_img, rh_mask, label = load_data(data_pair)
            data_batch.append([grasp_img, arm_img, rh_mask])
            label_batch.append(label)
            if i_step%BATCH_SIZE==0:
                grasp_img_batch = np.array([grasp_img for grasp_img, arm_img, rh_mask in data_batch])
                arm_img_batch = np.array([arm_img for grasp_img, arm_img, rh_mask in data_batch])
                rh_mask_batch = np.array([rh_mask for grasp_img, arm_img, rh_mask in data_batch])
                label_batch = np.array(label_batch, dtype=np.int)
                test_step([grasp_img_batch, arm_img_batch, rh_mask_batch], label_batch)
                data_batch, label_batch = [], []
            if i_step%LOG_STEP==0:
                print("test step - {}/{}        ".format(i_step, N_test), end = '\r')
        with test_summary_writer.as_default():
            tf.summary.scalar('loss', test_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

        model.save(model_log_dir.format(epoch + 1))

        print("")
        print("=================================================================")
        print(
            f'Epoch {epoch + 1}, '
            f'Loss: {train_loss.result()}, '
            f'Accuracy: {train_accuracy.result() * 100}, '
            f'Test Loss: {test_loss.result()}, '
            f'Test Accuracy: {test_accuracy.result() * 100}'
        )
        print("=================================================================")
        print("")
        res_list.append((float(test_accuracy.result())*100, float(test_loss.result())))
    return res_list

## train filtered data

In [8]:
ROBOT_MODEL_ROOT_DIV = "{}-filtered".format(ROBOT_MODEL_ROOT)
data_pairs_train_div, data_pairs_test_div = \
        data_pairs_train_filtered, data_pairs_test_filtered

res_list = train_script(data_pairs_train_div, data_pairs_test_div, ROBOT_MODEL_ROOT_DIV,
                        EPOCHS_S=0, EPOCHS_E=20)
print("{} acc/loss list:".format(ROBOT_MODEL_ROOT_DIV))
for i_r, res in enumerate(res_list):
    print("{}: {} %, {} ".format(i_r+1, np.round(res[0],2), np.round(res[1], 3)))

Log path: /home/rnb/Projects/rnb-planning/model/latticized/panda-filtered/20211004-024301


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-filtered/20211004-024301/model_1/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-filtered/20211004-024301/model_1/assets
/home/rnb/.local/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)



Epoch 1, Loss: 0.4142005443572998, Accuracy: 82.75499725341797, Test Loss: 0.4340701997280121, Test Accuracy: 85.29647827148438



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-filtered/20211004-024301/model_2/assets


...

INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-filtered/20211004-024301/model_19/assets
/home/rnb/.local/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)



Epoch 19, Loss: 0.09017126262187958, Accuracy: 96.97000122070312, Test Loss: 0.4752272963523865, Test Accuracy: 86.02764892578125



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-filtered/20211004-024301/model_20/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-filtered/20211004-024301/model_20/assets
/home/rnb/.local/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)



Epoch 20, Loss: 0.06805729866027832, Accuracy: 97.65999603271484, Test Loss: 0.45358923077583313, Test Accuracy: 84.16465759277344

/home/rnb/Projects/rnb-planning/model/latticized/panda-filtered acc/loss list:
1: 85.3 %, 0.434 
2: 86.34 %, 0.325 
3: 86.72 %, 0.335 
4: 86.9 %, 0.343 
5: 86.76 %, 0.32 
6: 86.98 %, 0.31 
7: 86.57 %, 0.32 
8: 86.56 %, 0.331 
9: 87.35 %, 0.316 
10: 87.47 %, 0.313 
11: 86.67 %, 0.353 
12: 86.72 %, 0.318 
13: 86.87 %, 0.323 
14: 85.3 %, 0.362 
15: 86.09 %, 0.358 
16: 85.38 %, 0.494 
17: 83.89 %, 0.481 
18: 86.11 %, 0.386 
19: 86.03 %, 0.475 
20: 84.16 %, 0.454 


```
16: 83.57 %, 1.115 
17: 87.54 %, 0.303 
18: 87.14 %, 0.319 
19: 86.62 %, 0.338 
20: 86.24 %, 0.373 
```

## failmore data

In [9]:
ROBOT_MODEL_ROOT_DIV = "{}-failmore".format(ROBOT_MODEL_ROOT)
data_pairs_train_div, data_pairs_test_div = \
        data_pairs_train_failmore, data_pairs_test_failmore

res_list = train_script(data_pairs_train_div, data_pairs_test_div, ROBOT_MODEL_ROOT_DIV,
                        EPOCHS_S=0, EPOCHS_E=20)
print("{} acc/loss list:".format(ROBOT_MODEL_ROOT_DIV))
for i_r, res in enumerate(res_list):
    print("{}: {} %, {} ".format(i_r+1, np.round(res[0],2), np.round(res[1], 3)))

Log path: /home/rnb/Projects/rnb-planning/model/latticized/panda-failmore/20211004-030343


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-failmore/20211004-030343/model_1/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-failmore/20211004-030343/model_1/assets
/home/rnb/.local/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)



Epoch 1, Loss: 0.3022109568119049, Accuracy: 91.28999328613281, Test Loss: 0.16595174372196198, Test Accuracy: 95.91346740722656



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-failmore/20211004-030343/model_2/assets


...

INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-failmore/20211004-030343/model_19/assets
/home/rnb/.local/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)



Epoch 19, Loss: 0.019637124612927437, Accuracy: 99.4000015258789, Test Loss: 0.10061381012201309, Test Accuracy: 96.45433044433594



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-failmore/20211004-030343/model_20/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-failmore/20211004-030343/model_20/assets
/home/rnb/.local/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)



Epoch 20, Loss: 0.01980873942375183, Accuracy: 99.31999969482422, Test Loss: 0.11305000633001328, Test Accuracy: 96.85496520996094

/home/rnb/Projects/rnb-planning/model/latticized/panda-failmore acc/loss list:
1: 95.91 %, 0.166 
2: 96.23 %, 0.1 
3: 97.01 %, 0.076 
4: 96.79 %, 0.074 
5: 96.27 %, 0.078 
6: 96.99 %, 0.081 
7: 96.35 %, 0.08 
8: 95.46 %, 0.096 
9: 95.37 %, 0.092 
10: 96.15 %, 0.089 
11: 96.52 %, 0.102 
12: 96.79 %, 0.095 
13: 96.34 %, 0.094 
14: 96.7 %, 0.09 
15: 97.04 %, 0.106 
16: 96.9 %, 0.107 
17: 96.53 %, 0.102 
18: 97.03 %, 0.107 
19: 96.45 %, 0.101 
20: 96.85 %, 0.113 


## balanced data

In [10]:
ROBOT_MODEL_ROOT_DIV = "{}-balanced".format(ROBOT_MODEL_ROOT)
data_pairs_train_div, data_pairs_test_div = \
        data_pairs_train_mixed, data_pairs_test_mixed

res_list = train_script(data_pairs_train_div, data_pairs_test_div, ROBOT_MODEL_ROOT_DIV,
                        EPOCHS_S=0, EPOCHS_E=20)
print("{} acc/loss list:".format(ROBOT_MODEL_ROOT_DIV))
for i_r, res in enumerate(res_list):
    print("{}: {} %, {} ".format(i_r+1, np.round(res[0],2), np.round(res[1], 3)))

Log path: /home/rnb/Projects/rnb-planning/model/latticized/panda-balanced/20211004-032424


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-balanced/20211004-032424/model_1/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-balanced/20211004-032424/model_1/assets
/home/rnb/.local/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)



Epoch 1, Loss: 0.1871277093887329, Accuracy: 94.22000122070312, Test Loss: 0.14328347146511078, Test Accuracy: 95.20232391357422



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-balanced/20211004-032424/model_2/assets


...

INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-balanced/20211004-032424/model_19/assets
/home/rnb/.local/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)



Epoch 19, Loss: 0.047069523483514786, Accuracy: 98.91500091552734, Test Loss: 0.2041892409324646, Test Accuracy: 94.61137390136719



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-balanced/20211004-032424/model_20/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-balanced/20211004-032424/model_20/assets
/home/rnb/.local/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)



Epoch 20, Loss: 0.016879646107554436, Accuracy: 99.48999786376953, Test Loss: 0.22836506366729736, Test Accuracy: 94.85176086425781

/home/rnb/Projects/rnb-planning/model/latticized/panda-balanced acc/loss list:
1: 95.2 %, 0.143 
2: 94.96 %, 0.191 
3: 94.76 %, 0.166 
4: 95.16 %, 0.166 
5: 94.8 %, 0.176 
6: 95.0 %, 0.185 
7: 94.81 %, 0.182 
8: 94.92 %, 0.222 
9: 94.12 %, 0.216 
10: 94.78 %, 0.221 
11: 95.04 %, 0.226 
12: 94.69 %, 0.266 
13: 94.67 %, 0.27 
14: 94.72 %, 0.238 
15: 95.03 %, 0.242 
16: 95.15 %, 0.225 
17: 95.07 %, 0.29 
18: 94.69 %, 0.241 
19: 94.61 %, 0.204 
20: 94.85 %, 0.228 


# Train results - Panda

#### filtered: 20211004-024301
```
1: 85.3 %, 0.434 
2: 86.34 %, 0.325 
3: 86.72 %, 0.335 
4: 86.9 %, 0.343 
5: 86.76 %, 0.32 
6: 86.98 %, 0.31 
7: 86.57 %, 0.32 
8: 86.56 %, 0.331 
9: 87.35 %, 0.316 
10: 87.47 %, 0.313 
11: 86.67 %, 0.353 
12: 86.72 %, 0.318 
13: 86.87 %, 0.323 
14: 85.3 %, 0.362 
15: 86.09 %, 0.358 
16: 85.38 %, 0.494 
17: 83.89 %, 0.481 
18: 86.11 %, 0.386 
19: 86.03 %, 0.475 
20: 84.16 %, 0.454 
```

#### failmore: 20211004-030343
```
1: 95.91 %, 0.166 
2: 96.23 %, 0.1 
3: 97.01 %, 0.076 
4: 96.79 %, 0.074 
5: 96.27 %, 0.078 
6: 96.99 %, 0.081 
7: 96.35 %, 0.08 
8: 95.46 %, 0.096 
9: 95.37 %, 0.092 
10: 96.15 %, 0.089 
11: 96.52 %, 0.102 
12: 96.79 %, 0.095 
13: 96.34 %, 0.094 
14: 96.7 %, 0.09 
15: 97.04 %, 0.106 
16: 96.9 %, 0.107 
17: 96.53 %, 0.102 
18: 97.03 %, 0.107 
19: 96.45 %, 0.101 
20: 96.85 %, 0.113 
```

#### 20211004-032424
```
1: 95.2 %, 0.143 
2: 94.96 %, 0.191 
3: 94.76 %, 0.166 
4: 95.16 %, 0.166 
5: 94.8 %, 0.176 
6: 95.0 %, 0.185 
7: 94.81 %, 0.182 
8: 94.92 %, 0.222 
9: 94.12 %, 0.216 
10: 94.78 %, 0.221 
11: 95.04 %, 0.226 
12: 94.69 %, 0.266 
13: 94.67 %, 0.27 
14: 94.72 %, 0.238 
15: 95.03 %, 0.242 
16: 95.15 %, 0.225 
17: 95.07 %, 0.29 
18: 94.69 %, 0.241 
19: 94.61 %, 0.204 
20: 94.85 %, 0.228 
```

## Load & test

In [11]:
def get_tf_functions(model_root, model_folder, epoch_folder):
    # last_model = '20210606-204309' # panda
    # last_model = '20210610-052216' # indy
    # last_model = sorted(os.listdir(ROBOT_MODEL_ROOT))[-1]
    logpath = os.path.join(model_root, model_folder)

    # epochlist = sorted([item for item in os.listdir(logpath) if item.startswith("model")], key=lambda x: int(x[6:]))
    # print(epochlist)
    # last_save = epochlist[-1]
    model_log_dir = os.path.join(logpath, epoch_folder)

    import tensorflow as tf
    model = tf.keras.models.load_model(model_log_dir)

    @tf.function
    def inference(images):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=False)
        return predictions

    loss_object = tf.keras.losses.BinaryCrossentropy()

    @tf.function
    def calc_loss(labels, predictions):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        return loss_object(labels, predictions)
    return model, inference, calc_loss

def test_script(data_pairs_test, model_root, model_folder, epoch_folder):
    BATCH_SIZE = 1
    LOG_STEP = 100
    N_test = len(data_pairs_test)
    gtimer = GlobalTimer.instance()
    gtimer.reset()
    
    model, inference, calc_loss= get_tf_functions(model_root, model_folder, epoch_folder)

    grasp_img, arm_img, rh_mask, label = load_data(data_pairs_test[0])
    res = inference([np.array([grasp_img]), np.array([arm_img]), np.array([rh_mask])])


    i_step = 0
    res_list = []
    label_list = []
    loss_list= []
    grasp_img_batch = []
    arm_img_batch = []
    rh_mask_batch = []
    label_batch = [] 
    for data_pair in data_pairs_test:
        i_step += 1
        grasp_img, arm_img, rh_mask, label = load_data(data_pair)
        grasp_img_batch.append(grasp_img)
        arm_img_batch.append(arm_img)
        rh_mask_batch.append(rh_mask)
        label_batch.append(label)
        if len(grasp_img_batch)==BATCH_SIZE:
            grasp_img_batch, arm_img_batch, rh_mask_batch = np.array(grasp_img_batch), np.array(arm_img_batch), np.array(rh_mask_batch)
            with gtimer.block("inference"):
                res = inference([grasp_img_batch, arm_img_batch, rh_mask_batch])
            loss = calc_loss(label_batch, res)
            res_list = res_list + list(res.numpy()>0.5)
            label_list = label_list + label_batch
            loss_list.append(loss.numpy())
            grasp_img_batch = []
            arm_img_batch = []
            rh_mask_batch = []
            label_batch = [] 
        if i_step%LOG_STEP==0:
            print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

    res_list = np.array(res_list)[:,1]
    label_list = np.array(label_list)[:,1]
    loss_list = np.array(loss_list)[:]

    acc = np.mean(np.equal(res_list, label_list)) * 100
    mean_loss = np.mean(loss_list)

    print("")
    print("=================================================================")
    print(
        f'Test Loss: {mean_loss} \n'
        f'Test Accuracy: {acc} \n'
        f'TP / FN / ACC: {np.sum(np.logical_and(res_list, label_list))}, ' 
        f'{np.sum(np.logical_and(np.logical_not(res_list), label_list))}, ' 
        f'{round(np.mean(res_list[np.where(label_list)])*100,2)}\n'
        f'FP / TN / ACC: {np.sum(np.logical_and(res_list, np.logical_not(label_list)))}, '
        f'{np.sum(np.logical_and(np.logical_not(res_list), np.logical_not(label_list)))}, '
        f'{round(np.mean(np.logical_not(res_list[np.where(np.logical_not(label_list))]))*100,2)}\n'
        f'PACC / NACC / TACC: {round(np.mean(label_list[np.where(res_list)])*100,2)}, '
        f'{round(np.mean(np.logical_not(label_list[np.where(np.logical_not(res_list))]))*100,2)}, '
        f'{round(np.mean(res_list==label_list)*100,2)}\n'
    )
    print("=================================================================")
    print("")
    print(gtimer)

    TP_FN_FP_TN = (np.sum(np.logical_and(res_list, label_list)), np.sum(np.logical_and(np.logical_not(res_list), label_list)), 
                   np.sum(np.logical_and(res_list, np.logical_not(label_list))), np.sum(np.logical_and(np.logical_not(res_list), np.logical_not(label_list))))
        
    return label_list, res_list

# Test all

In [12]:
print("========= Filtered =========")
model_foler, epoch_folder = '20211004-024301', 'model_10'
filtered_res = []
filtered_res.append(test_script(data_pairs_test_filtered, ROBOT_MODEL_ROOT+"-filtered", model_foler, epoch_folder))
filtered_res.append(test_script(data_pairs_test_failmore, ROBOT_MODEL_ROOT+"-filtered", model_foler, epoch_folder))
filtered_res.append(test_script(data_pairs_test_mixed, ROBOT_MODEL_ROOT+"-filtered", model_foler, epoch_folder))

print("========= Failmore =========")
model_foler, epoch_folder = '20211004-030343', 'model_15'
failmore_res = []
failmore_res.append(test_script(data_pairs_test_filtered, ROBOT_MODEL_ROOT+"-failmore", model_foler, epoch_folder))
failmore_res.append(test_script(data_pairs_test_failmore, ROBOT_MODEL_ROOT+"-failmore", model_foler, epoch_folder))
failmore_res.append(test_script(data_pairs_test_mixed, ROBOT_MODEL_ROOT+"-failmore", model_foler, epoch_folder))

print("========= Balanced =========")
model_foler, epoch_folder = '20211004-032424', 'model_1'
balanced_res = []
balanced_res.append(test_script(data_pairs_test_filtered, ROBOT_MODEL_ROOT+"-balanced", model_foler, epoch_folder))
balanced_res.append(test_script(data_pairs_test_failmore, ROBOT_MODEL_ROOT+"-balanced", model_foler, epoch_folder))
balanced_res.append(test_script(data_pairs_test_mixed, ROBOT_MODEL_ROOT+"-balanced", model_foler, epoch_folder))

print("========= Filtered =========")
print("filtered:")
print_confusion_mat(*filtered_res[0])
print("failmore:")
print_confusion_mat(*filtered_res[1])
print("balanced:")
print_confusion_mat(*filtered_res[2])
print("========================\n")

print("========= Failmore =========")
print("filtered:")
print_confusion_mat(*failmore_res[0])
print("failmore:")
print_confusion_mat(*failmore_res[1])
print("balanced:")
print_confusion_mat(*failmore_res[2])
print("========================\n")

print("========= Balanced =========")
print("filtered:")
print_confusion_mat(*balanced_res[0])
print("failmore:")
print_confusion_mat(*balanced_res[1])
print("balanced:")
print_confusion_mat(*balanced_res[2])
print("========================\n")

========= Filtered =========


test step - 5000/5000        
Test Loss: 0.3125072717666626 
Test Accuracy: 87.28 
TP / FN / ACC: 2916, 256, 91.93
FP / TN / ACC: 380, 1448, 79.21
PACC / NACC / TACC: 88.47, 84.98, 87.28


inference: 	58047.0 ms/5000 = 11.609 ms (10.975/41.927)



test step - 5000/5000        
Test Loss: 0.7413970232009888 
Test Accuracy: 69.58 
TP / FN / ACC: 183, 21, 89.71
FP / TN / ACC: 1500, 3296, 68.72
PACC / NACC / TACC: 10.87, 99.37, 69.58


inference: 	57126.0 ms/5000 = 11.425 ms (10.824/13.939)



test step - 5000/5000        
Test Loss: 0.49009305238723755 
Test Accuracy: 80.42 
TP / FN / ACC: 2294, 206, 91.76
FP / TN / ACC: 773, 1727, 69.08
PACC / NACC / TACC: 74.8, 89.34, 80.42


inference: 	56482.0 ms/5000 = 11.296 ms (10.843/14.05)

========= Failmore =========


test step - 5000/5000        
Test Loss: 1.311219334602356 
Test Accuracy: 66.84 
TP / FN / ACC: 1650, 1522, 52.02
FP / TN / ACC: 136, 1692, 92.56
PACC / NACC / TACC: 92.39, 52.64, 66.84


inference: 	56694.0 ms/5000 = 11.339 ms (10.842/15.513)



test step - 5000/5000        
Test Loss: 0.10572824627161026 
Test Accuracy: 97.06 
TP / FN / ACC: 104, 100, 50.98
FP / TN / ACC: 47, 4749, 99.02
PACC / NACC / TACC: 68.87, 97.94, 97.06


inference: 	57510.0 ms/5000 = 11.502 ms (10.947/20.012)



test step - 5000/5000        
Test Loss: 1.00685453414917 
Test Accuracy: 75.22 
TP / FN / ACC: 1293, 1207, 51.72
FP / TN / ACC: 32, 2468, 98.72
PACC / NACC / TACC: 97.58, 67.16, 75.22


inference: 	56743.0 ms/5000 = 11.349 ms (10.859/16.608)

========= Balanced =========


test step - 5000/5000        
Test Loss: 0.744101345539093 
Test Accuracy: 72.52 
TP / FN / ACC: 3133, 39, 98.77
FP / TN / ACC: 1335, 493, 26.97
PACC / NACC / TACC: 70.12, 92.67, 72.52


inference: 	56545.0 ms/5000 = 11.309 ms (10.867/15.335)



test step - 5000/5000        
Test Loss: 0.20582441985607147 
Test Accuracy: 92.2 
TP / FN / ACC: 201, 3, 98.53
FP / TN / ACC: 387, 4409, 91.93
PACC / NACC / TACC: 34.18, 99.93, 92.2


inference: 	56743.0 ms/5000 = 11.349 ms (10.777/18.38)



test step - 5000/5000        
Test Loss: 0.14306873083114624 
Test Accuracy: 95.19999999999999 
TP / FN / ACC: 2469, 31, 98.76
FP / TN / ACC: 209, 2291, 91.64
PACC / NACC / TACC: 92.2, 98.66, 95.2


inference: 	57094.0 ms/5000 = 11.419 ms (10.826/40.461)

========= Filtered =========
filtered:
	 PP 	 	 PN 	 	 5000
GP 	 2916 	 	 256 	 	 91.93%
GN 	 380 	 	 1448 	 79.21%
AL 	 88.47% 	 84.98% 	 87.28%
failmore:
	 PP 	 	 PN 	 	 5000
GP 	 183 	 	 21 	 	 89.71%
GN 	 1500 	 	 3296 	 68.72%
AL 	 10.87% 	 99.37% 	 69.58%
balanced:
	 PP 	 	 PN 	 	 5000
GP 	 2294 	 	 206 	 	 91.76%
GN 	 773 	 	 1727 	 69.08%
AL 	 74.80% 	 89.34% 	 80.42%

========= Failmore =========
filtered:
	 PP 	 	 PN 	 	 5000
GP 	 1650 	 	 1522 	 	 52.02%
GN 	 136 	 	 1692 	 92.56%
AL 	 92.39% 	 52.64% 	 66.84%
failmore:
	 PP 	 	 PN 	 	 5000
GP 	 104 	 	 100 	 	 50.98%
GN 	 47 	 	 4749 	 99.02%
AL 	 68.87% 	 97.94% 	 97.06%
balanced:
	 PP 	 	 PN 	 	 5000
GP 	 1293 	 	 1207 	 	 51.72%
GN 	 32 	 	 2468 	 98.72%
AL 	 97.58% 	 6

### Test result - panda
```
inference: 	57094.0 ms/5000 = 11.419 ms (10.826/40.461)

========= Filtered =========
filtered:
         PP 	 	      PN 	 	 5000
GP 	 2916 	 	    256 	 	91.93%
GN 	 380 	 	   1448 	   79.21%
AL 	 88.47% 	84.98%      87.28%
failmore:
	 PP 	 	 PN 	 	 5000
GP 	 183 	 	 21 	 	 89.71%
GN 	 1500 	 	 3296 	 68.72%
AL 	 10.87% 	 99.37% 	 69.58%
balanced:
	 PP 	 	 PN 	 	 5000
GP 	 2294 	 	 206 	 	 91.76%
GN 	 773 	 	 1727 	 69.08%
AL 	 74.80% 	 89.34% 	 80.42%
========================

========= Failmore =========
filtered:
	 PP 	 	 PN 	 	 5000
GP 	 1650 	 	 1522 	 	 52.02%
GN 	 136 	 	 1692 	 92.56%
AL 	 92.39% 	 52.64% 	 66.84%
failmore:
	 PP 	 	 PN 	 	 5000
GP 	 104 	 	 100 	 	 50.98%
GN 	 47 	 	 4749 	 99.02%
AL 	 68.87% 	 97.94% 	 97.06%
balanced:
	 PP 	 	 PN 	 	 5000
GP 	 1293 	 	 1207 	 	 51.72%
GN 	 32 	 	 2468 	 98.72%
AL 	 97.58% 	 67.16% 	 75.22%
========================

========= Balanced =========
filtered:
	 PP 	 	 PN 	 	 5000
GP 	 3133 	 	 39 	 	 98.77%
GN 	 1335 	 	 493 	 26.97%
AL 	 70.12% 	 92.67% 	 72.52%
failmore:
	 PP 	 	 PN 	 	 5000
GP 	 201 	 	 3 	 	 98.53%
GN 	 387 	 	 4409 	 91.93%
AL 	 34.18% 	 99.93% 	 92.20%
balanced:
	 PP 	 	 PN 	 	 5000
GP 	 2469 	 	 31 	 	 98.76%
GN 	 209 	 	 2291 	 91.64%
AL 	 92.20% 	 98.66% 	 95.20%
========================
```

## Test convergence

In [ ]:
last_model = sorted(os.listdir(ROBOT_MODEL_ROOT))[-1]
logpath = os.path.join(ROBOT_MODEL_ROOT, last_model)

model_epoch_list = []
acc_epoch_list = []
loss_epoch_list = []
# last_save = sorted([item for item in os.listdir(logpath) if item.startswith("model")])[-1]
# last_save = 'model_1'
for last_save in sorted([item for item in os.listdir(logpath) if item.startswith("model")], key=lambda x: int(x[6:])):
    model_log_dir = os.path.join(logpath, last_save)

    import tensorflow as tf
    model = tf.keras.models.load_model(model_log_dir)

    @tf.function
    def inference(images):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=False)
        return predictions

    loss_object = tf.keras.losses.BinaryCrossentropy()

    @tf.function
    def calc_loss(labels, predictions):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        return loss_object(labels, predictions)
    
    BATCH_SIZE = 50
    LOG_STEP = 100
    N_test = len(data_pairs_test)
    gtimer = GlobalTimer.instance()
    gtimer.reset()

    i_step = 0
    res_list = []
    label_list = []
    loss_list= []
    grasp_img_batch = []
    arm_img_batch = []
    rh_mask_batch = []
    label_batch = [] 
    for data_pair in data_pairs_test:
        i_step += 1
        grasp_img, arm_img, rh_mask, label = load_data(data_pair)
        grasp_img_batch.append(grasp_img)
        arm_img_batch.append(arm_img)
        rh_mask_batch.append(rh_mask)
        label_batch.append(label)
        if len(grasp_img_batch)==BATCH_SIZE:
            grasp_img_batch, arm_img_batch, rh_mask_batch = np.array(grasp_img_batch), np.array(arm_img_batch), np.array(rh_mask_batch)
            with gtimer.block("inference"):
                res = inference([grasp_img_batch, arm_img_batch, rh_mask_batch])
            loss = calc_loss(label_batch, res)
            res_list = res_list + list(res.numpy()>0.5)
            label_list = label_list + label_batch
            loss_list.append(loss.numpy())
            grasp_img_batch = []
            arm_img_batch = []
            rh_mask_batch = []
            label_batch = [] 
        if i_step%LOG_STEP==0:
            print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

    res_list = np.array(res_list)[:5000,1]
    label_list = np.array(label_list)[:5000,1]
    loss_list = np.array(loss_list)[:5000]
    
    acc = np.mean(np.equal(res_list, label_list)) * 100
    mean_loss = np.mean(loss_list)

    print("")
    print("=================================================================")
    print(
        f'Test Loss: {mean_loss} \n'
        f'Test Accuracy: {acc} \n'
        f'TP / FN / ACC: {np.sum(np.logical_and(res_list, label_list))}, ' 
        f'{np.sum(np.logical_and(np.logical_not(res_list), label_list))}, ' 
        f'{round(np.mean(res_list[np.where(label_list)])*100,2)}\n'
        f'FP / TN / ACC: {np.sum(np.logical_and(res_list, np.logical_not(label_list)))}, '
        f'{np.sum(np.logical_and(np.logical_not(res_list), np.logical_not(label_list)))}, '
        f'{round(np.mean(np.logical_not(res_list[np.where(np.logical_not(label_list))]))*100,2)}\n'
        f'PACC / NACC / TACC: {round(np.mean(label_list[np.where(res_list)])*100,2)}, '
        f'{round(np.mean(np.logical_not(label_list[np.where(np.logical_not(res_list))]))*100,2)}, '
        f'{round(np.mean(res_list==label_list)*100,2)}\n'
    )
    print("=================================================================")
    print("")
    print(gtimer)
    model_epoch_list.append(last_save)
    acc_epoch_list.append(acc)
    loss_epoch_list.append(mean_loss)

In [ ]:
import matplotlib.pyplot as plt
print(model_epoch_list)
plt.figure(figsize=(10,3))
plt.subplot(1,2,1)
plt.plot(acc_epoch_list)
plt.subplot(1,2,2)
plt.plot(loss_epoch_list)

In [ ]:
save_json(ROBOT_TYPE_NAME+".json", {"epoch": np.array(model_epoch_list), "acc": np.array(acc_epoch_list), "loss": np.array(loss_epoch_list)})

## test on shared array

In [ ]:
import SharedArray as sa
BATCH_SIZE = 1
SERVER_PERIOD = 1e-2
# Create an array in shared memory.
robot_type_p = sa.attach("shm://robot_type")
grasp_img_p = sa.attach("shm://grasp_img")
arm_img_p = sa.attach("shm://arm_img")
rh_mask_p = sa.attach("shm://rh_mask")
result_p = sa.attach("shm://result")
query_in = sa.attach("shm://query_in")
response_out = sa.attach("shm://response_out")
query_quit = sa.attach("shm://query_quit")

def query_wait_response(grasp_img_batch, arm_img_batch, rh_mask_batch):
    grasp_img_p[:] = grasp_img_batch[:]
    arm_img_p[:] = arm_img_batch[:]
    rh_mask_p[:] = rh_mask_batch[:]
    query_in[0] = True
    while not response_out[0]:
        time.sleep(SERVER_PERIOD)
    response_out[0] = False
    return np.copy(result_p)

def quit_shared_server():
    query_quit[0] = True

In [ ]:
LOG_STEP = 100
N_test = len(data_pairs_test)

i_step = 0
data_batch, label_batch = [], []
result_list = []
label_list = []
for data_pair in data_pairs_test:
    i_step += 1
    grasp_img, arm_img, rh_mask, label = load_data(data_pair)
    data_batch.append([grasp_img, arm_img, rh_mask])
    label_batch.append(label)
    if i_step%BATCH_SIZE==0:
        grasp_img_batch = np.array([grasp_img for grasp_img, arm_img, rh_mask in data_batch])
        arm_img_batch = np.array([arm_img for grasp_img, arm_img, rh_mask in data_batch])
        rh_mask_batch = np.array([rh_mask for grasp_img, arm_img, rh_mask in data_batch])
        label_batch = np.array(label_batch, dtype=np.int)
        result = query_wait_response(grasp_img_batch, arm_img_batch, rh_mask_batch)
        result_list.append(result)
        label_list.append(label_batch)
        data_batch, label_batch = [], []
    if i_step%LOG_STEP==0:
        print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

In [ ]:
result_all = np.array(result_list)
label_all = np.array(label_list)

In [ ]:
np.mean(np.round(result_all).astype(np.int) == label_all.astype(np.int), axis=0)

In [ ]:
quit_shared_server()